In [7]:
import re
from tqdm import tqdm_notebook
import json

In [39]:
with open('../done/kuruch.json', encoding='utf-8') as f:
    kuruch_dict = json.load(f)

In [40]:
kuruch_dict[0]

{'cyrillic_backwards': 'а',
 'transcription_backwards': 'a',
 'cyrillic': 'а',
 'transcription': 'a',
 'derived_cyrillic': 'а',
 'derived_transcription': 'a',
 'form_cyrillic': '',
 'form_transcription': '',
 'pos': '',
 'common_number': '',
 'type': '',
 'type_alternation': '',
 'definitions': ['а', '\\[я\\] приду не сегодня, а завтра'],
 'examples': [{'example_cyrillic': 'пуада эмм та̄рьм, а ента',
   'example_transcription': "puada emm taar'm, a jenta",
   'example_definition': ''},
  {'example_cyrillic': 'мунн лӣннче пӯдзэ пынньенҍ, а со̄нн ля̄ннч лӣхемь лаппьсъенҍ',
   'example_transcription': "munn l'iinnče puudze pɨɲ'ɲ'jen', a soonn l'aannč l'iix'em' lap'p'sjen'",
   'example_definition': 'я буду оленеводом, а она будет дояркой;'},
  {'example_cyrillic': 'а ке̄ тоннӭ цӣӆькэ тэнн баяс?',
   'example_transcription': "a k'ee ton'n'e c'iil̥'ke tenn bajas?",
   'example_definition': 'а кто тебе сказал об этом?'}]}

In [41]:
def list_from_letters(word):
    letter_list = []
    n_for_list = 0
    for n, letter in enumerate(word):
        if letter == "'" or letter == "̥":
            if word[n-1] == "'" or word[n-1] == "̥":
                letter_list[n_for_list - 1] = word[n-2:n+1]
            else:
                letter_list[n_for_list - 1] = word[n-1:n+1]                 
        else:
            letter_list.append(letter)
            n_for_list += 1 
    n = 1
    for letter in letter_list[1:]:
        if letter == letter_list[n-1]:
            letter_list[n-1] = letter + letter
            del letter_list[n]
        else:
            n += 1
    n = 1
    for letter in letter_list[1:]:
        if letter == 'a' or letter == 'e':
            if letter_list[n-1] == 'o' or letter_list[n-1] == 'u':
                letter_list[n-1] += letter
                del letter_list[n]
            else:
                n += 1
        else:
                n += 1
    return letter_list

In [42]:
def minimal_pairs_dict(word_list):
    min_pairs = {}
    for n_word1,word1 in tqdm_notebook(enumerate(word_list)):
        for word2 in word_list[n_word1 + 1:]:
            letters_word1 = list_from_letters(word1)
            letters_word2 = list_from_letters(word2)
            if len(letters_word1) == len(letters_word2):
                ndiff = 0
                for n, letter1 in enumerate(letters_word1):
                    letter2 = letters_word2[n]
                    if letter1 != letter2:
                        ndiff += 1
                        word1_letter = letter1
                        word2_letter = letter2
                if ndiff==1:
                    word1, word2 = "", ""
                    for letter in letters_word1:
                        word1 += letter
                    for letter in letters_word2:
                        word2 += letter
                    if word1 in min_pairs:
                        min_pairs[word1].append([word2, word1_letter, word2_letter])
                    else:
                        min_pairs[word1] = [[word2, word1_letter, word2_letter]]
    return min_pairs

In [43]:
min_pairs = minimal_pairs_dict([x['transcription'] for x in kuruch_dict])

In [44]:
kuruch_dict_search = {word['transcription']:word for word in kuruch_dict}

In [45]:
with open("../done/min_pairs.csv", 'w', encoding="utf-8") as f:
    f.write('word1;word1 definitions;word2;word2 definitions;word1 sound; word2 sound\n')
    for word, pairs in min_pairs.items():
        for pair in pairs:
            line = f"{word};{kuruch_dict_search[word]['definitions']};{pair[0]};{kuruch_dict_search[pair[0]]['definitions']};{pair[1]};{pair[2]}\n"
            f.write(line)